In [1]:
import sys, time, pickle, torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
sys.path.insert(0, '../../Preprocess')
import numpy as np
import pandas as pd
from preload_models import get_sst2_tok_n_model
from _utils import sample_random_glue_sst2, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    attr_normalizing_func, \
                    collect_info_for_metric, save_info

In [2]:
sst2_data_raw, targets, idxs = sample_random_glue_sst2()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3b24abff24d1d8c0.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5960909ab3834668.arrow


In [3]:
tokenizer, model = get_sst2_tok_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import Lime
from captum._utils.models.linear_model import SkLearnLasso
from captum.attr import visualization 

In [6]:
lime = Lime(model, interpretable_model=SkLearnLasso(alpha=0.000003))

In [7]:
def generate_record(raw_review, target):
    #tokenizer operations
    tokenized = tokenizer(raw_review, truncation=True, return_offsets_mapping=True)
    offset_mapping = tokenized['offset_mapping']
    conti_map = get_continuation_mapping(offset_mapping)
    input_ids = torch.tensor(tokenized['input_ids']).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    print(f'deotk {detokenized}')
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    
    #categorizing results
    pred_class = 'Pos' if pred_prob > 0.5 else 'Neg' 
    true_class = 'Pos' if target > 0.5 else 'Neg' 
    
    #attribution algorithm working
    attribution = lime.attribute(input_emb, n_samples=3000, show_progress=True, perturbations_per_eval=1000)
    attribution[torch.isnan(attribution)] = 0
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Pos' if attr_score > 0.5 else 'Neg'
    convergence_score = None
   
    attr_score = torch.sum(word_attributions)
    attr_class = 'Pos' if attr_score > 0.5 else 'Neg'
    convergence_score = None
    
    if torch.all(torch.isnan(attr_score)):
        print(f'nan attri score')
        word_attributions = torch.zeros(word_attributions.size())
    
    #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)
    print(f'word attri {word_attributions}')
    print(f'conti attri {conti_attr}')
#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
#     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(sst2_data_raw, targets), start=1):
    print(f'Raw review: {datum_raw}')
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))
   

Raw review: its oscar nomination 
GT target: 1
deotk ['<s>', 'its', 'o', 'scar', 'nomination', '', '</s>']


Lime attribution: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.36s/it]


word attr tensor([0.7472, 0.5716, 0.0567, 0.0338, 0.1816, 0.2786, 0.0049])
conti attr [tensor(0.7472), tensor(0.5716), tensor(0.0905), tensor(0.1816), tensor(0.2786), tensor(0.0049)]
detokenized ['<s>', 'its', 'o', 'scar', 'nomination', '', '</s>']
len conti_raw 6
conti_raw ['<s>', 'its', 'oscar', 'nomination', '', '</s>']
word attri tensor([0.7472, 0.5716, 0.0567, 0.0338, 0.1816, 0.2786, 0.0049])
conti attri tensor([0.7472, 0.5716, 0.0905, 0.1816, 0.2786, 0.0049])


/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.965e-02, tolerance: 1.942e-02
  model = cd_fast.enet_coordinate_descent(


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Pos,Pos (1.00),Pos,1.87,#s its oscar nomination #/s


...

Lime attribution: 100%|███████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.71s/it]


word attr tensor([ 0.6333, -0.3093,  0.0200, -0.3292, -0.6276, -0.0264])
conti attr [tensor(0.6333), tensor(-0.6185), tensor(-0.6276), tensor(-0.0264)]
detokenized ['<s>', 'old', '-', 'hat', '', '</s>']
len conti_raw 4
conti_raw ['<s>', 'old-hat', '', '</s>']
word attri tensor([ 0.6333, -0.3093,  0.0200, -0.3292, -0.6276, -0.0264])
conti attri tensor([ 0.6333, -0.6185, -0.6276, -0.0264])


/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:645: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.297e-04, tolerance: 1.975e-04
  model = cd_fast.enet_coordinate_descent(


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.00),Neg,-0.64,#s old-hat #/s


<IPython.core.display.HTML object>


In [9]:
save_info(idxs, sst2_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='lime_out.pkl')

{'indices': [27798,
  28244,
  48083,
  8795,
  91,
  20682,
  59890,
  41763,
  19795,
  9957,
  28334,
  55664,
  16438,
  18031,
  9297,
  66913,
  9289,
  6789,
  25294,
  33294,
  29699,
  41960,
  6554,
  8772,
  17921,
  8027,
  49091,
  54186,
  15080,
  22382,
  23856,
  9464,
  23831,
  59434,
  17862,
  24584,
  26216,
  63094,
  55787,
  3993,
  25463,
  17540,
  51128,
  46224,
  32656,
  30105,
  28646,
  17011,
  7812,
  48236],
 'raw_data': ['its oscar nomination ',
  'shenanigans and slapstick ',
  'an unsettling sight , ',
  'the climactic hourlong cricket match ',
  'alternating between facetious comic parody and pulp melodrama , this smart-aleck movie ... tosses around some intriguing questions about the difference between human and android life ',
  'to be a part of that elusive adult world ',
  'emotional power ',
  'reminds you of why animation is such a perfect medium for children , because of the way it allows the mind to enter and accept another world ',
  'un

In [10]:
len(idxs)

50